In [1]:
import pymorphy2
import nltk
import ujson as json
import matplotlib.pyplot as plt
import numpy as np
import itertools
import gzip
from scipy.spatial.distance import cosine, pdist

import cowsay
from random import choice

from copy import deepcopy

nltk.download('punkt')

from datetime import datetime

from collections import Counter, defaultdict

from gensim.models import word2vec

from nltk.corpus import stopwords

nltk.download('stopwords')

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /home/a3nippo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/a3nippo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
class Document:
    def __init__(self, init_dict):
        self.title = init_dict.get('title', '')
        self.description = init_dict.get('description', '')
        self.url = init_dict.get('url', '')
        self.site = init_dict.get('site', '')
        self.ts = datetime.fromtimestamp(init_dict['ts']) if 'ts' in init_dict else -1
    
    def __str__(self):
        res = ''
        res += 'url : %s\n' % self.url
        res += 'date : %s\n' % self.ts
        res += 'title : %s\n' % self.title
        res += 'description : %s\n' % self.description
        res += 'site : %s\n' % self.site
        return res
    
characters = [
    'beavis', 
    'cheese',
    'daemon',
    'cow',
    'dragon',
    'ghostbusters',
    'kitty',
    'meow',
    'milk',
    'stegosaurus',
    'stimpy',
    'turkey',
    'turtle',
    'tux'
]

In [3]:
fin = gzip.open('dataset_mai.jsonl.gz')
for line in itertools.islice(fin, 10):
    data = json.loads(line.strip())
    print(Document(data))

url : http://bloknot-volzhsky.ru/news/volzhane-mogut-podat-zayavlenie-na-letnie-putevki-
date : 2019-11-30 18:26:10
title : Волжане могут подать заявление на летние путевки для детей
description : С понедельника заявления начинают принимать в МФЦ
site : bloknot-volzhsky.ru

url : https://trikky.ru/test-na-znanie-russkogo-yazyka-423354.html
date : 2019-11-30 18:26:48
title : 💗Тест на знание русского языка💗
description : Тест со сложными и легкими вопросами. Для кого-то будет легко набрать все 100 баллов, а кому-то будет немного тяжело. В любом случае попробовать стоит.1. Что изучает фразеология? способы образования слов устойчивые сочетания слов части речи2. На месте каких цифр в словах пишется н? В простенке между занавеше(1)ыми окнами были установле(2)ы часы, а рядом с ними […]
site : trikky.ru

url : https://topwar.ru/165315-chernomorskij-flot-popolnilsja-150-tonnym-plavkranom-proekta-02690.html
date : 2019-11-30 18:26:43
title : Черноморский флот пополнился 150-тонным плавкраном про

In [4]:
fin = gzip.open('dataset_mai.jsonl.gz')
dataset = []
dataset_test = []
for line in itertools.islice(fin, 10000):
    data = json.loads(line.strip())
    dataset.append(Document(data))

## ДЗ - реализовать поиск похожих документов по текстовым векторам и по word2vec векторам

# Реализация

In [5]:
%%time

morth_analyzer = pymorphy2.MorphAnalyzer()
        

functors_pos = {
    'INTJ',
    'PRCL',
    'CONJ',
    'PREP',
    'PRED',
    'NPRO'
}


MORTH_CACHE = {}
def get_norm_word_v4(a_word):
    analyzed = morth_analyzer.parse(a_word)[0]
    
    if a_word not in MORTH_CACHE:
        if (analyzed in stopwords.words("russian")) or (analyzed.tag.POS in functors_pos) \
            or (a_word in stopwords.words("russian")):
            MORTH_CACHE[a_word] = None
        else:
            MORTH_CACHE[a_word] = analyzed.normal_form
        
    
    return MORTH_CACHE[a_word]


def split_words_v3(a_text):
    cur_word = ''
    prev_is_alpha = False

    for letter in a_text:
        if  (letter.isalpha() and prev_is_alpha or 
            letter.isdigit() and not prev_is_alpha):
            cur_word += letter
        elif (letter.isalpha() and not prev_is_alpha or
             letter.isdigit() and prev_is_alpha):
            if cur_word: yield cur_word
            cur_word = letter
            prev_is_alpha = not prev_is_alpha
        else:
            if cur_word: yield cur_word
            cur_word = ''
            prev_is_alpha = False
    if cur_word: yield cur_word

        
def get_doc_words(a_doc, a_split=split_words_v3, a_norm_word=get_norm_word_v4):
    for word in itertools.chain(a_split(a_doc.title), a_split(a_doc.description)):
        
        normalized = a_norm_word(word)
        
        if normalized is not None:
            yield normalized


words_of_documents = []
all_words = set()

filtered_dataset = []

for doc in dataset:
    words = list(get_doc_words(doc))
    
    words = list(filter(lambda x: len(x) > 2, words))[:100]
    
    if not words:
        continue
    
    if not hasattr(doc, 'words'):
        doc.words = words
    
    filtered_dataset.append(doc)
    
    all_words.update(words)
    
    uniq_words = Counter(words)
    words_of_documents.append(uniq_words)
    
dataset = filtered_dataset
    
doc2words = dict(zip(dataset, words_of_documents))

all_words = list(all_words)

words_indices = {}

for i, word in enumerate(all_words):
    words_indices[word] = i

CPU times: user 1min 48s, sys: 1.65 s, total: 1min 50s
Wall time: 1min 50s


In [6]:
def init_idf(a_dataset, a_doc2words=doc2words, a_all_words=all_words):
    """calc inverted document frequency"""
    docs_count = len(a_dataset)
    
    idf = [0] * len(a_all_words)
    
    for i, word in enumerate(a_all_words):
        for doc in a_dataset:
            if a_doc2words[doc][word] > 0:
                idf[i] += 1
        
        idf[i] = np.log(docs_count / idf[i])
        
    return idf

idf = init_idf(dataset)

In [7]:
class TooRareError(Exception):
    def __str__(self):
        return "Document content is too rare"


def print_header(func_name):
    print()
    
    say = f'Function: {func_name}'
    
    getattr(cowsay, choice(characters))(say)
    
    print()

    
def construct_vector(a_doc, a_doc2words=doc2words, a_all_words=all_words):
    if hasattr(a_doc, 'regular_vector'):
        if a_doc.regular_vector is not None:
            return deepcopy(a_doc.regular_vector)
    
    counted_a_doc_words = a_doc2words[a_doc]
    
    a_doc_vec = np.array([0] * len(a_all_words))
    
    for i, el in enumerate(a_all_words):
        a_doc_vec[i] = counted_a_doc_words[el]
    
    a_doc.regular_vector = a_doc_vec
        
    return deepcopy(a_doc_vec)


def construct_LSA_vector(a_doc, a_idf=idf):
    if hasattr(a_doc, 'lsa_vector'):
        if a_doc.lsa_vector is not None:
            return deepcopy(a_doc.lsa_vector)
    
    a_doc_vec = construct_vector(a_doc)
    
    words_count = a_doc_vec.sum()
    
    a_doc_vec = a_doc_vec / words_count * np.array(a_idf)
    
    a_doc.lsa_vector = a_doc_vec
    
    return deepcopy(a_doc_vec)


def construct_w2v_mean_vector(a_doc, a_word2vec_model, a_idf=idf, a_words_indices=words_indices):
    # idf is taken into account
    a_doc_words = list(doc2words[a_doc].keys())
    
    suitable_words = [word for word in a_doc_words if word in a_word2vec_model.wv.vocab]
    
    if not suitable_words:
        raise TooRareError
    
    a_doc_vecs = a_word2vec_model[suitable_words]
    
    for word, i in zip(a_doc_words, range(len(a_doc_vecs))):
        a_doc_vecs[i] *= a_idf[a_words_indices[word]]
        
    return np.mean(a_doc_vecs, axis=0)


def print_results(a_doc, a_most_similar_docs):
    original_doc = 'Original Doc:' + '\n' + str(a_doc)
    
    print(original_doc)
    
    for doc, similarity in a_most_similar_docs:
        print(f'Similarity: {similarity}')
        print(doc)

        
def get_word_match_most_similar_docs(a_doc, a_dataset, a_top_n=10):
    print_header(get_word_match_most_similar_docs.__name__)
    
    a_doc_vec = construct_vector(a_doc)
    
    cosines = Counter()
    
    for doc in a_dataset:
        doc_vec = construct_vector(doc)
        
        # strange cosine, look documentation
        cos = cosine(a_doc_vec, doc_vec)
        
        if np.isnan(cos).any():
            continue
            
        cosines[doc] = 1 - cos
        
#         differences[doc] = -abs(a_doc_vec - doc_vec).sum()
        
    most_similar_docs = cosines.most_common(a_top_n)
    
    print_results(a_doc, most_similar_docs)
    

def get_tf_idf_most_similar_doc(a_doc, a_dataset, a_top_n=10):
    print_header(get_tf_idf_most_similar_doc.__name__)
    
    a_doc_vec = construct_LSA_vector(a_doc)
    
    # cosines between a_doc and doc from a_dataset
    cosines = Counter()
    
    for doc in a_dataset:
        doc_vec = construct_LSA_vector(doc)
        # strange cosine, look documentation
        cos = cosine(a_doc_vec, doc_vec)
        
        if np.isnan(cos).any():
            continue
            
        cosines[doc] = 1 - cos
    
    most_similar_docs = cosines.most_common(a_top_n)
    
    print_results(a_doc, most_similar_docs)

def get_w2v_most_similar_doc(a_doc, a_dataset, a_top_n=10):
    print_header(get_w2v_most_similar_doc.__name__)
    
    for doc in dataset:
        if not hasattr(doc, 'words'):
            doc.words = list(get_doc_words(doc))
    
    sentences = [doc.words for doc in a_dataset]
    
#     sentences = [doc.words for doc in a_dataset]
    
    w2v = word2vec.Word2Vec(sentences, workers=8, size = 500, iter=10, alpha=0.1, min_count=2)
    
    w2v.init_sims(replace=True)
    
    try:
        a_doc_vec = construct_w2v_mean_vector(a_doc, w2v, idf)
    except TooRareError as err:
        print(err)
        return

    a_doc_vec = construct_w2v_mean_vector(a_doc, w2v)
    
    cosines = Counter()
    
    for doc in a_dataset:
        try:
            doc_vec = construct_w2v_mean_vector(doc, w2v)
        except TooRareError:
            continue
        
        cos = cosine(a_doc_vec, doc_vec)
        
        if np.isnan(cos).any():
            continue
            
        cosines[doc] = 1 - cos
#         cosines[doc] = -pdist([a_doc_vec, doc_vec])[0]
        
    most_similar_docs = cosines.most_common(a_top_n)
    
    print_results(a_doc, most_similar_docs)

## Пример формата выдачи

In [8]:
# get_ololo_most_similar_doc(dataset[13], w2v_dataset, dataset)

#### Тестовые данные

In [9]:
doc_id = 13 #dota2
doc_id = 1946 #гороскоп
doc_id = 3388 #хоккей
doc_id = 7601 #телефоны

In [10]:
%%time

doc_id = 13 
for similart_func in [get_word_match_most_similar_docs,  get_tf_idf_most_similar_doc, get_w2v_most_similar_doc]:
    similart_func(dataset[doc_id], dataset)


  __________________________________________
< Function: get_word_match_most_similar_docs >
                                               \
                                                \
                                                 \
                                                  \                                        
                                                                                      ___-------___
                                                                                  _-~~             ~~-_
                                                                               _-~                    /~-_
                                                    /^\__/^\         /~  \                   /    \
                                                  /|  O|| O|        /      \_______________/        \
                                                 | |___||__|      /       /                \          \
                                                

Original Doc:
url : https://cyber.sports.ru/dota2/1080755627.html
date : 2019-11-30 18:26:39
title : Результаты Parimatch League Dota 2. Virtus.pro победила
description : 30 ноября завершился турнир Parimatch League. В финале Virtus.pro разгромила HellRaisers со счетом 3:0 и заработала 40 тысяч долларов. Лан-финал Parimatch League прошел с 28 по 30 ноября в Москве. 4 команды разыграли 70 тысяч долларов. Результаты команд 1. Virtus.pro2.
site : cyber.sports.ru

Similarity: 1.0
url : https://cyber.sports.ru/dota2/1080755627.html
date : 2019-11-30 18:26:39
title : Результаты Parimatch League Dota 2. Virtus.pro победила
description : 30 ноября завершился турнир Parimatch League. В финале Virtus.pro разгромила HellRaisers со счетом 3:0 и заработала 40 тысяч долларов. Лан-финал Parimatch League прошел с 28 по 30 ноября в Москве. 4 команды разыграли 70 тысяч долларов. Результаты команд 1. Virtus.pro2.
site : cyber.sports.ru

Similarity: 0.8330833177897394
url : https://cyber.sports.ru/dota2/1

In [11]:
doc_id = 1946 
for similart_func in [get_word_match_most_similar_docs,  get_tf_idf_most_similar_doc, get_w2v_most_similar_doc]:
    similart_func(dataset[doc_id], dataset)


  __________________________________________
< Function: get_word_match_most_similar_docs >
                                               \
                                                \
                                                 \
                                                  \
                                                                 _ ___.--'''`--''//-,-_--_.
                                                     \`"' ` || \\ \ \\/ / // / ,-\\`,_
                                                    /'`  \ \ || Y  | \|/ / // / - |__ `-,
                                                   /\@"\  ` \ `\ |  | ||/ // | \/  \  `-._`-,_.,
                                                  /  _.-. `.-\,___/\ _/|_/_\_\/|_/ |     `-._._)
                                                  `-'``/  /  |  // \__/\__  /  \__/ \
                                                       `-'  /-\/  | -|   \__ \   |-' |
                                                         __/\ / _/ \/

Original Doc:
url : https://www.obozrevatel.com/astro/news/goroskop-na-1-dekabrya-chto-zhdet-lvov-rakov-dev-i-drugie-znaki-zodiaka.htm
date : 2019-11-30 17:07:11
title : Гороскоп на 1 декабря: что ждет Львов, Раков, Дев и другие знаки зодиака
description : Советы астрологов на воскресенье
site : obozrevatel.com

Similarity: 1.0
url : https://www.obozrevatel.com/astro/news/goroskop-na-1-dekabrya-chto-zhdet-lvov-rakov-dev-i-drugie-znaki-zodiaka.htm
date : 2019-11-30 17:07:11
title : Гороскоп на 1 декабря: что ждет Львов, Раков, Дев и другие знаки зодиака
description : Советы астрологов на воскресенье
site : obozrevatel.com

Similarity: 0.8169580101966858
url : https://sterlegrad.ru/newsrb/118119-nazvany-znaki-zodiaka-chya-zhizn-kruto-izmenitsya-v-dekabre-2019-goda.html
date : 2019-11-30 10:46:26
title : Названы знаки Зодиака, чья жизнь круто изменится в декабре 2019 года
description : По словам астрологов, в декабре фортуна будет благосклонна ко многим представителям зодиакальных Знаков,

In [12]:
doc_id = 3388 
for similart_func in [get_word_match_most_similar_docs,  get_tf_idf_most_similar_doc, get_w2v_most_similar_doc]:
    similart_func(dataset[doc_id], dataset)


  __________________________________________
< Function: get_word_match_most_similar_docs >
                                               \
                                                \
                                                 \
                                                  \
                                              
                                                  ("`-'  '-/") .___..--' ' "`-._
                                                      ` *_ *  )    `-.   (      ) .`-.__. `)
                                                      (_Y_.) ' ._   )   `._` ;  `` -. .-'
                                                   _.. `--'_..-_/   /--' _ .' ,4
                                                ( i l ),-''  ( l i),'  ( ( ! .-'  
                                                     
                                                     

Original Doc:
url : https://www.tv21.ru/news/2019/11/30/khokkeisty-murmana-proveli-pervyy-domashniy-match-na-stadione-str

Original Doc:
url : https://www.tv21.ru/news/2019/11/30/khokkeisty-murmana-proveli-pervyy-domashniy-match-na-stadione-stroitel
date : 2019-11-30 15:31:47
title : Хоккеисты "Мурмана" провели первый домашний матч на стадионе "Строитель"
description : Игра состоялась в рамках открытия нового сезона Суперлиги по хоккею с мячом.
site : tv21.ru

Similarity: 1.0
url : https://www.tv21.ru/news/2019/11/30/khokkeisty-murmana-proveli-pervyy-domashniy-match-na-stadione-stroitel
date : 2019-11-30 15:31:47
title : Хоккеисты "Мурмана" провели первый домашний матч на стадионе "Строитель"
description : Игра состоялась в рамках открытия нового сезона Суперлиги по хоккею с мячом.
site : tv21.ru

Similarity: 0.9016067981719971
url : https://severpost.ru/read/87561
date : 2019-11-30 17:32:40
title : Первый домашний матч «Мурман» сыграл вничью
description : В своем первом домашнем матче «Мурман» не смог одолеть «Волгу». Игра проходила на стадионе «Строитель». В первом тайме мурманчане уступили хоккеистам из

In [13]:
doc_id = 7601
for similart_func in [get_word_match_most_similar_docs,  get_tf_idf_most_similar_doc, get_w2v_most_similar_doc]:
    similart_func(dataset[doc_id], dataset)


  __________________________________________
< Function: get_word_match_most_similar_docs >
                                               \
                                                \
                                                 \
                                                  \                                        
                                                              __---__
                                                           _-       /--______
                                                      __--( /     \ )XXXXXXXXXXX\v.
                                                    .-XXX(   O   O  )XXXXXXXXXXXXXXX-
                                                   /XXX(       U     )        XXXXXXX\
                                                 /XXXXX(              )--_  XXXXXXXXXXX\
                                                /XXXXX/ (      O     )   XXXXXX   \XXXXX\
                                                XXXXX/   /            XXXXXX   \_

Original Doc:
url : https://megaobzor.com/Stala-izvestna-cena-smartfona-Redmi-K30.html
date : 2019-11-30 12:12:16
title : Стала известна цена смартфона Redmi K30
description : Авторитетный искатель утечек Мукул Шарма поделился подробностями о цене смартфона Redmi K30, официальный анонс которого состоится уже 10 декабря. Если верить источнику, аппарат обойдется в 327 долларов, что намного больше 285 долларов, которые ему приписывали ранее. По предварительным данным, Redmi K30 получит аккумулятор ёмкостью 5000 мАч, квадрокамеру с главным датчиком изображения разрешением 60 Мп, дисплей с частотой обновления 120 Гц, двойную фронтальную камеру на 20 Мп и 2 Мп и восьмиядерный процессор Snapdragon 730G.
site : megaobzor.com

Similarity: 1.0
url : https://megaobzor.com/Stala-izvestna-cena-smartfona-Redmi-K30.html
date : 2019-11-30 12:12:16
title : Стала известна цена смартфона Redmi K30
description : Авторитетный искатель утечек Мукул Шарма поделился подробностями о цене смартфона Redmi K30, о

Original Doc:
url : https://megaobzor.com/Stala-izvestna-cena-smartfona-Redmi-K30.html
date : 2019-11-30 12:12:16
title : Стала известна цена смартфона Redmi K30
description : Авторитетный искатель утечек Мукул Шарма поделился подробностями о цене смартфона Redmi K30, официальный анонс которого состоится уже 10 декабря. Если верить источнику, аппарат обойдется в 327 долларов, что намного больше 285 долларов, которые ему приписывали ранее. По предварительным данным, Redmi K30 получит аккумулятор ёмкостью 5000 мАч, квадрокамеру с главным датчиком изображения разрешением 60 Мп, дисплей с частотой обновления 120 Гц, двойную фронтальную камеру на 20 Мп и 2 Мп и восьмиядерный процессор Snapdragon 730G.
site : megaobzor.com

Similarity: 1.0
url : https://megaobzor.com/Stala-izvestna-cena-smartfona-Redmi-K30.html
date : 2019-11-30 12:12:16
title : Стала известна цена смартфона Redmi K30
description : Авторитетный искатель утечек Мукул Шарма поделился подробностями о цене смартфона Redmi K30, о